# THIS IS A JULIA (.jl) LANGUAGE NOTEBOOK
## Install Julia & Import needed packages

In [20]:
# Call the packages you are going to use
using Plots
#using Flux
using DataFrames
using CSV
using BenchmarkTools
#using StatsBase
#using MultivariateStats
#using DelimitedFiles

In [57]:
training_dataset = "train.csv"
testing_dataset =  "test.csv"

"test.csv"

## Read the training dataset ("train.csv") as a DataFrame
## Then, display the first 10 rows to get a feel for the data

In [22]:
@time df_training = CSV.read(training_dataset, DataFrame)
@time first(df_training,10)

  0.006690 seconds (11.87 k allocations: 619.391 KiB)
  0.000078 seconds (128 allocations: 13.766 KiB)


Row,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,String1,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64?,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64?,Int64
1,000ff2bfdfe9,0.209377,3109.03,85.2001,22.3944,8.13869,0.699861,0.025578,9.81221,5.55563,4126.59,22.5984,175.639,152.708,823.928,257.432,47.2234,0.563481,23.3876,4.85192,0.023482,1.05022,0.069225,13.7841,1.30201,36.206,69.0834,295.571,0.23868,0.284232,89.2456,84.3166,29.6571,5.31069,1.74307,23.1877,7.29418,1.98728,1433.17,0.949104,B,30.8794,78.527,3.82838,13.3946,10.2651,9028.29,3.58345,7.29816,1.73855,0.094822,11.3391,72.6111,2003.81,22.1362,69.8349,0.120343,1
2,007255e47698,0.145282,978.764,85.2001,36.9689,8.13869,3.63219,0.025578,13.5178,1.2299,5496.93,19.4205,155.868,14.7547,51.2169,257.432,30.2843,0.48471,50.6282,6.08504,0.031442,1.11387,1.1178,28.311,1.35718,37.4766,70.7984,178.553,0.23868,0.363489,110.582,75.7455,37.532,0.0055176,1.74307,17.2223,4.9264,0.858603,1111.29,0.003042,A,109.125,95.4151,52.2605,17.176,0.29685,6785.0,10.3589,0.173229,0.49706,0.568932,9.2927,72.6111,27981.6,29.1354,32.132,21.978,0
3,013f2bd269f5,0.47003,2635.11,85.2001,32.3606,8.13869,6.73284,0.025578,12.8246,1.2299,5135.78,26.4825,128.989,219.32,482.142,257.432,32.5637,0.495852,85.9554,5.37649,0.036218,1.05022,0.70035,39.3647,1.00961,21.4596,70.8197,321.427,0.23868,0.210441,120.056,65.4698,28.0535,1.28974,1.74307,36.8614,7.81367,8.14665,1494.08,0.377208,B,109.125,78.527,5.39063,224.207,8.7452,8338.91,11.6269,7.70956,0.97556,1.19882,37.0778,88.6094,13677.0,28.0229,35.1927,0.196941,0
4,043ac50845d5,0.252107,3819.65,120.202,77.1122,8.13869,3.68534,0.025578,11.0537,1.2299,4169.68,23.6577,237.282,11.0504,661.519,257.432,15.2019,0.717882,88.1594,2.34765,0.029054,1.4003,0.636075,41.117,0.722727,21.5304,47.2759,196.608,0.23868,0.292431,139.825,71.5712,24.3549,2.65535,1.74307,52.0039,7.38606,3.81333,15691.6,0.614484,B,31.6744,78.527,31.3234,59.302,7.88434,10965.8,14.852,6.12216,0.49706,0.284466,18.5296,82.4168,2094.26,39.9487,90.4932,0.155829,0
5,044fb8a146ec,0.380297,3733.05,85.2001,14.1037,8.13869,3.94225,0.05481,3.39678,102.152,5728.73,24.0108,324.546,149.717,6074.86,257.432,82.2135,0.536467,72.6443,30.5377,0.025472,1.05022,0.69315,31.7247,0.82755,34.4154,74.0653,200.178,0.23868,0.207708,97.9201,52.8389,26.0199,1.1449,1.74307,9.06486,7.35072,3.49085,1403.66,0.164268,B,109.125,91.9948,51.1413,29.1026,4.27464,16198.0,13.6667,8.15306,48.5013,0.121914,16.4087,146.11,8524.37,45.3813,36.2626,0.0966145,1
6,04517a3c90bd,0.209377,2615.81,85.2001,8.54153,8.13869,4.01313,0.025578,12.5473,1.2299,5237.54,10.2399,148.488,16.5261,642.325,257.432,18.382,0.63946,80.6674,14.688,0.016716,1.05022,0.857625,32.457,1.39028,7.03064,55.224,135.489,0.23868,0.478275,135.318,81.4631,31.7316,0.0055176,1.74307,16.7731,4.9264,2.39441,866.383,0.003042,A,109.125,78.527,3.82838,23.305,0.29685,8517.28,10.9819,0.173229,0.49706,1.16496,21.9155,72.6111,24177.6,28.5252,82.5278,21.978,0
7,049232ca8356,0.348249,1733.65,85.2001,8.37739,15.3125,1.91354,0.025578,6.54778,1.2299,5710.46,17.655,143.647,344.644,719.725,257.432,38.4551,0.946323,78.3049,13.1843,0.033631,1.05022,0.61095,13.7841,2.78608,21.8775,19.2157,107.908,1.31801,0.46051,176.626,97.0759,44.5061,1.00696,1.74307,4.47403,4.9264,2.62015,1793.61,0.097344,B,13.2145,78.527,26.3049,48.2664,1.4605,3903.81,10.7779,4.40848,0.8613,0.467337,17.8784,192.453,3332.47,34.1662,100.087,0.0650959,0
8,057287f2da6d,0.269199,966.455,85.2001,21.1742,8.13869,4.98762,0.025578,9.40889,1.2299,5040.78,20.8329,170.052

In [23]:
training_data_columns = names(df_training)

58-element Vector{String}:
 "Id"
 "AB"
 "AF"
 "AH"
 "AM"
 "AR"
 "AX"
 "AY"
 "AZ"
 "BC"
 "BD "
 "BN"
 "BP"
 ⋮
 "FE"
 "FI"
 "FL"
 "FR"
 "FS"
 "GB"
 "GE"
 "GF"
 "GH"
 "GI"
 "GL"
 "Class"

In [24]:
#Convert binary columns into integers
df_training_clean = df_training
df_training_clean.EJ = copy(replace.(df_training.EJ, "A" => 1.0))
df_training_clean.EJ = copy(replace.(df_training_clean.EJ, "B" => 2.0))


#Replace missing values with 0
for col in eachcol(df_training_clean)
    col = replace!(col, missing => 0)
end


#Remove the 1st "Id" Column
df_training_clean = select!(df_training_clean, Not(1))
print("df_training_clean is loaded")

df_training_clean is loaded

In [25]:
CSV.write("training_clean.csv", df_training_clean, delim = "|")

"training_clean.csv"

In [26]:
#import Pkg; Pkg.add("LIBSVM")
#Pkg.add("RDatasets")
using DelimitedFiles
clean_data = readdlm("training_clean.csv", '|', Float64, skipstart=1)

using LIBSVM, RDatasets
using LinearAlgebra, Random, Statistics

X = Matrix(clean_data[1:end,1:56])'
X_labels = Vector(clean_data[1:end,57])

617-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [27]:
function perclass_splits(y, percent)
    uniq_class = unique(y)
    keep_index = []
    for class in uniq_class
        class_index = findall(y .== class)
        row_index = randsubseq(class_index, percent)
        push!(keep_index, row_index...)
    end
    return keep_index
end

perclass_splits (generic function with 1 method)

In [28]:
Random.seed!(1)

train_index = perclass_splits(X_labels, 0.80)

test_index = setdiff(1:length(X_labels), train_index)

118-element Vector{Any}:
  17
  18
  25
  27
  29
  32
  36
  52
  60
  63
  64
  69
  79
   ⋮
 556
 563
 578
 580
 583
 587
 593
 602
 609
 610
 613
 617

In [29]:
#FEATURE SCALING -----> standardization (z-score normalization)
X = Matrix(clean_data[1:end,1:56])

#1. Make a new vector for all columns
f1 = X[:, :1]
f2 = X[:, :2]
f3 = X[:, :3]
f4 = X[:, :4]
f5 = X[:, :5]
f6 = X[:, :6]
f7 = X[:, :7]
f8 = X[:, :8]
f9 = X[:, :9]
f10 = X[:, :10]
f11 = X[:, :11]
f12 = X[:, :12]
f13 = X[:, :13]
f14 = X[:, :14]
f15 = X[:, :15]
f16 = X[:, :16]
f17 = X[:, :17]
f18 = X[:, :18]
f19 = X[:, :19]
f20 = X[:, :20]
f21 = X[:, :21]
f22 = X[:, :22]
f23 = X[:, :23]
f24 = X[:, :24]
f25 = X[:, :25]
f26 = X[:, :26]
f27 = X[:, :27]
f28 = X[:, :28]
f29 = X[:, :29]
f30 = X[:, :30]
f31 = X[:, :31]
f32 = X[:, :32]
f33 = X[:, :33]
f34 = X[:, :34]
f35 = X[:, :35]
f36 = X[:, :36]
f37 = X[:, :37]
f38 = X[:, :38]
f39 = X[:, :39]
f40 = X[:, :40]
f41 = X[:, :41]
f42 = X[:, :42]
f43 = X[:, :43]
f44 = X[:, :44]
f45 = X[:, :45]
f46 = X[:, :46]
f47 = X[:, :47]
f48 = X[:, :48]
f49 = X[:, :49]
f50 = X[:, :50]
f51 = X[:, :51]
f52 = X[:, :52]
f53 = X[:, :53]
f54 = X[:, :54]
f55 = X[:, :55]
f56 = X[:, :56]

#2. Make a mean value for each column vector
f1_bar = mean(f1)
f2_bar = mean(f2)
f3_bar = mean(f3)
f4_bar = mean(f4)
f5_bar = mean(f5)
f6_bar = mean(f6)
f7_bar = mean(f7)
f8_bar = mean(f8)
f9_bar = mean(f9)
f10_bar = mean(f10)
f11_bar = mean(f11)
f12_bar = mean(f12)
f13_bar = mean(f13)
f14_bar = mean(f14)
f15_bar = mean(f15)
f16_bar = mean(f16)
f17_bar = mean(f17)
f18_bar = mean(f18)
f19_bar = mean(f19)
f20_bar = mean(f20)
f21_bar = mean(f21)
f22_bar = mean(f22)
f23_bar = mean(f23)
f24_bar = mean(f24)
f25_bar = mean(f25)
f26_bar = mean(f26)
f27_bar = mean(f27)
f28_bar = mean(f28)
f29_bar = mean(f29)
f30_bar = mean(f30)
f31_bar = mean(f31)
f32_bar = mean(f32)
f33_bar = mean(f33)
f34_bar = mean(f34)
f35_bar = mean(f35)
f36_bar = mean(f36)
f37_bar = mean(f37)
f38_bar = mean(f38)
f39_bar = mean(f39)
f40_bar = mean(f40)
f41_bar = mean(f41)
f42_bar = mean(f42)
f43_bar = mean(f43)
f44_bar = mean(f44)
f45_bar = mean(f45)
f46_bar = mean(f46)
f47_bar = mean(f47)
f48_bar = mean(f48)
f49_bar = mean(f49)
f50_bar = mean(f50)
f51_bar = mean(f51)
f52_bar = mean(f52)
f53_bar = mean(f53)
f54_bar = mean(f54)
f55_bar = mean(f55)
f56_bar = mean(f56)

#3. Make a sigma value for each column vector
f1_std = std(f1)
f2_std = std(f2)
f3_std = std(f3)
f4_std = std(f4)
f5_std = std(f5)
f6_std = std(f6)
f7_std = std(f7)
f8_std = std(f8)
f9_std = std(f9)
f10_std = std(f10)
f11_std = std(f11)
f12_std = std(f12)
f13_std = std(f13)
f14_std = std(f14)
f15_std = std(f15)
f16_std = std(f16)
f17_std = std(f17)
f18_std = std(f18)
f19_std = std(f19)
f20_std = std(f20)
f21_std = std(f21)
f22_std = std(f22)
f23_std = std(f23)
f24_std = std(f24)
f25_std = std(f25)
f26_std = std(f26)
f27_std = std(f27)
f28_std = std(f28)
f29_std = std(f29)
f30_std = std(f30)
f31_std = std(f31)
f32_std = std(f32)
f33_std = std(f33)
f34_std = std(f34)
f35_std = std(f35)
f36_std = std(f36)
f37_std = std(f37)
f38_std = std(f38)
f39_std = std(f39)
f40_std = std(f40)
f41_std = std(f41)
f42_std = std(f42)
f43_std = std(f43)
f44_std = std(f44)
f45_std = std(f45)
f46_std = std(f46)
f47_std = std(f47)
f48_std = std(f48)
f49_std = std(f49)
f50_std = std(f50)
f51_std = std(f51)
f52_std = std(f52)
f53_std = std(f53)
f54_std = std(f54)
f55_std = std(f55)
f56_std = std(f56)

f1_s = (f1 .- f1_bar) ./ f1_std
f2_s = (f2 .- f2_bar) ./ f2_std
f3_s = (f3 .- f3_bar) ./ f3_std
f4_s = (f4 .- f4_bar) ./ f4_std
f5_s = (f5 .- f5_bar) ./ f5_std
f6_s = (f6 .- f6_bar) ./ f6_std
f7_s = (f7 .- f7_bar) ./ f7_std
f8_s = (f8 .- f8_bar) ./ f8_std
f9_s = (f9 .- f9_bar) ./ f9_std
f10_s = (f10 .- f10_bar) ./ f10_std
f11_s = (f11 .- f11_bar) ./ f11_std
f12_s = (f12 .- f12_bar) ./ f12_std
f13_s = (f13 .- f13_bar) ./ f13_std
f14_s = (f14 .- f14_bar) ./ f14_std
f15_s = (f15 .- f15_bar) ./ f15_std
f16_s = (f16 .- f16_bar) ./ f16_std
f17_s = (f17 .- f17_bar) ./ f17_std
f18_s = (f18 .- f18_bar) ./ f18_std
f19_s = (f19 .- f19_bar) ./ f19_std
f20_s = (f20 .- f20_bar) ./ f20_std
f21_s = (f21 .- f21_bar) ./ f21_std
f22_s = (f22 .- f22_bar) ./ f22_std
f23_s = (f23 .- f23_bar) ./ f23_std
f24_s = (f24 .- f24_bar) ./ f24_std
f25_s = (f25 .- f25_bar) ./ f25_std
f26_s = (f26 .- f26_bar) ./ f26_std
f27_s = (f27 .- f27_bar) ./ f27_std
f28_s = (f28 .- f28_bar) ./ f28_std
f29_s = (f29 .- f29_bar) ./ f29_std
f30_s = (f30 .- f30_bar) ./ f30_std
f31_s = (f31 .- f31_bar) ./ f31_std
f32_s = (f32 .- f32_bar) ./ f32_std
f33_s = (f33 .- f33_bar) ./ f33_std
f34_s = (f34 .- f34_bar) ./ f34_std
f35_s = (f35 .- f35_bar) ./ f35_std
f36_s = (f36 .- f36_bar) ./ f36_std
f37_s = (f37 .- f37_bar) ./ f37_std
f38_s = (f38 .- f38_bar) ./ f38_std
f39_s = (f39 .- f39_bar) ./ f39_std
f40_s = (f40 .- f40_bar) ./ f40_std
f41_s = (f41 .- f41_bar) ./ f41_std
f42_s = (f42 .- f42_bar) ./ f42_std
f43_s = (f43 .- f43_bar) ./ f43_std
f44_s = (f44 .- f44_bar) ./ f44_std
f45_s = (f45 .- f45_bar) ./ f45_std
f46_s = (f46 .- f46_bar) ./ f46_std
f47_s = (f47 .- f47_bar) ./ f47_std
f48_s = (f48 .- f48_bar) ./ f48_std
f49_s = (f49 .- f49_bar) ./ f49_std
f50_s = (f50 .- f50_bar) ./ f50_std
f51_s = (f51 .- f51_bar) ./ f51_std
f52_s = (f52 .- f52_bar) ./ f52_std
f53_s = (f53 .- f53_bar) ./ f53_std
f54_s = (f54 .- f54_bar) ./ f54_std
f55_s = (f55 .- f55_bar) ./ f55_std
f56_s = (f56 .- f56_bar) ./ f56_std

X = [f1_s f2_s f3_s f4_s f5_s f6_s f7_s f8_s f9_s f10_s f11_s f12_s f13_s f14_s f15_s f16_s f17_s f18_s f19_s f20_s f21_s f22_s f23_s f24_s f25_s f26_s f27_s f28_s f29_s f30_s f31_s f32_s f33_s f34_s f35_s f36_s f37_s f38_s f39_s f40_s f41_s f42_s f43_s f44_s f45_s f46_s f47_s f48_s f49_s f50_s f51_s f52_s f53_s f54_s f55_s f56_s ]'

56×617 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.571689   -0.70853    -0.0151997  …  -0.106427   -0.243269    0.0121686
 -0.170837   -1.09691    -0.376863      -0.453374   -0.973115   -0.360592
 -0.261457   -0.261457   -0.261457       0.090067   -0.261457    3.34827
 -0.237696   -0.028678   -0.0947679      0.235016   -0.219175    1.04746
 -0.189141   -0.189141   -0.189141      -0.0116638  -0.189141   -0.189141
 -1.89902    -0.749849    0.465284   …   0.989527    0.954809   -0.919968
 -0.0833498  -0.0833498  -0.0833498     -0.0833498  -0.0833498   0.135811
 -0.173361    0.678369    0.519032       1.11293    -0.699174   -1.64796
 -0.0383228  -0.104702   -0.104702      -0.104702   -0.104702   -0.00160117
 -0.405054    0.0485017  -0.0710312      0.178229   -0.275549   -0.838165
  0.338934   -0.574708    1.45561    …  -0.27016    -0.676223   -0.0671287
 -0.30264    -0.410094   -0.556184      -0.286213   -0.609588    0.406569
  0.664762   -0.769467    1.3573         0.158258   -0.9

In [30]:
# # Support-Vector Machine
################################################################################

Xtrain = X[:,train_index]

Xtest = X[:,test_index]

ytrain = X_labels[train_index]

ytest = X_labels[test_index]


# run model

model_radialbasis = svmtrain(Xtrain, ytrain, gamma=0.013, probability = true)
model_linear = svmtrain(Xtrain, ytrain, kernel =Kernel.Linear)
model_polynomial = svmtrain(Xtrain, ytrain, kernel =Kernel.Polynomial)
model_sigmoid = svmtrain(Xtrain, ytrain, kernel =Kernel.Sigmoid)



y_hat, decision_values = LIBSVM.svmpredict(model_radialbasis, Xtest)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0372582776670726 0.05222278588195151 … 0.05815180504557997 0.09272756962939435; 0.9627417223329274 0.9477772141180485 … 0.9418481949544201 0.9072724303706057])

In [31]:
# check accuracy
accuracy = mean(y_hat .== ytest)

0.9152542372881356

In [32]:
# display results

check = [y_hat[i] == ytest[i] for i in 1:length(y_hat)]

check_display = [y_hat ytest check]

118×3 Matrix{Float64}:
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  1.0  0.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 ⋮         
 0.0  1.0  0.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  1.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 0.0  0.0  1.0

In [33]:
y_hat, decision_values = LIBSVM.svmpredict(model_radialbasis, X)

([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.6276575116729863 0.009062112173863013 … 0.046339007452503744 0.09272756962939435; 0.37234248832701367 0.990937887826137 … 0.9536609925474963 0.9072724303706057])

In [34]:
summary = DataFrame([decision_values[1, :] decision_values[2, :] y_hat X_labels], :auto)
show(summary, allrows=true)

617×4 DataFrame
 Row │ x1          x2          x3       x4      
     │ Float64     Float64     Float64  Float64 
─────┼──────────────────────────────────────────
   1 │ 0.627658    0.372342        1.0      1.0
   2 │ 0.00906211  0.990938        0.0      0.0
   3 │ 0.0711344   0.928866        0.0      0.0
   4 │ 0.0712697   0.92873         0.0      0.0
   5 │ 0.537007    0.462993        1.0      1.0
   6 │ 0.0123259   0.987674        0.0      0.0
   7 │ 0.0399877   0.960012        0.0      0.0
   8 │ 0.0138954   0.986105        0.0      0.0
   9 │ 0.00489675  0.995103        0.0      0.0
  10 │ 0.0251883   0.974812        0.0      0.0
  11 │ 0.987226    0.0127742       1.0      1.0
  12 │ 0.0525826   0.947417        0.0      0.0
  13 │ 0.0439761   0.956024        0.0      0.0
  14 │ 0.90971     0.0902905       1.0      1.0
  15 │ 0.0600542   0.939946        0.0      0.0
  16 │ 0.137336    0.862664        0.0      0.0
  17 │ 0.0372583   0.962742        0.0      0.0
  18 │ 0.0522228   0.

In [35]:
insertcols!(summary, :Act_minus_Pred => summary[:,4] - summary[:,3])

Row,x1,x2,x3,x4,Act_minus_Pred
,Float64,Float64,Float64,Float64,Float64
1,0.627658,0.372342,1.0,1.0,0.0
2,0.00906211,0.990938,0.0,0.0,0.0
3,0.0711344,0.928866,0.0,0.0,0.0
4,0.0712697,0.92873,0.0,0.0,0.0
5,0.537007,0.462993,1.0,1.0,0.0
6,0.0123259,0.987674,0.0,0.0,0.0
7,0.0399877,0.960012,0.0,0.0,0.0
8,0.0138954,0.986105,0.0,0.0,0.0
9,0.00489675,0.995103,0.0,0.0,0.0


In [36]:
show(summary, allrows=true)

617×5 DataFrame
 Row │ x1          x2          x3       x4       Act_minus_Pred 
     │ Float64     Float64     Float64  Float64  Float64        
─────┼──────────────────────────────────────────────────────────
   1 │ 0.627658    0.372342        1.0      1.0             0.0
   2 │ 0.00906211  0.990938        0.0      0.0             0.0
   3 │ 0.0711344   0.928866        0.0      0.0             0.0
   4 │ 0.0712697   0.92873         0.0      0.0             0.0
   5 │ 0.537007    0.462993        1.0      1.0             0.0
   6 │ 0.0123259   0.987674        0.0      0.0             0.0
   7 │ 0.0399877   0.960012        0.0      0.0             0.0
   8 │ 0.0138954   0.986105        0.0      0.0             0.0
   9 │ 0.00489675  0.995103        0.0      0.0             0.0
  10 │ 0.0251883   0.974812        0.0      0.0             0.0
  11 │ 0.987226    0.0127742       1.0      1.0             0.0
  12 │ 0.0525826   0.947417        0.0      0.0             0.0
  13 │ 0.0439761   0.

In [37]:
using StatsBase
countmap(summary[!, :Act_minus_Pred])

Dict{Float64, Int64} with 3 entries:
  0.0  => 583
  -1.0 => 5
  1.0  => 29

# The following is like the above but all in one go, and using "test.csv" instead of "train.csv"

In [67]:
@time df_actual_test = CSV.read(testing_dataset, DataFrame)
@time first(df_actual_test,10)

#-------------------------------------------------------------
#%%julia
#Convert binary columns into integers
df_training_clean = df_actual_test
df_training_clean.EJ = copy(replace.(df_training_clean.EJ, "A" => 1.0))
df_training_clean.EJ = copy(replace.(df_training_clean.EJ, "B" => 2.0))


#Replace missing values with 0
for col in eachcol(df_training_clean)
    col = replace!(col, missing => 0)
end


#Remove the 1st "Id" Column
df_training_clean = select!(df_training_clean, Not(1))
print("df_training_clean is loaded")
#-----------------------------------------------------

CSV.write("test_clean.csv", df_training_clean, delim = "|")

#-----------------------------------------------------
clean_data = readdlm("test_clean.csv", '|', Float64, skipstart=1)

using LIBSVM, RDatasets
using LinearAlgebra, Random, Statistics
#-----------------------------------------------------

print("check1")

#FEATURE SCALING -----> standardization (z-score normalization)
X = Matrix(clean_data[1:end,1:56])

print("check2")

#1. Make a new vector for all columns
f1 = X[:, :1]
f2 = X[:, :2]
f3 = X[:, :3]
f4 = X[:, :4]
f5 = X[:, :5]
f6 = X[:, :6]
f7 = X[:, :7]
f8 = X[:, :8]
f9 = X[:, :9]
f10 = X[:, :10]
f11 = X[:, :11]
f12 = X[:, :12]
f13 = X[:, :13]
f14 = X[:, :14]
f15 = X[:, :15]
f16 = X[:, :16]
f17 = X[:, :17]
f18 = X[:, :18]
f19 = X[:, :19]
f20 = X[:, :20]
f21 = X[:, :21]
f22 = X[:, :22]
f23 = X[:, :23]
f24 = X[:, :24]
f25 = X[:, :25]
f26 = X[:, :26]
f27 = X[:, :27]
f28 = X[:, :28]
f29 = X[:, :29]
f30 = X[:, :30]
f31 = X[:, :31]
f32 = X[:, :32]
f33 = X[:, :33]
f34 = X[:, :34]
f35 = X[:, :35]
f36 = X[:, :36]
f37 = X[:, :37]
f38 = X[:, :38]
f39 = X[:, :39]
f40 = X[:, :40]
f41 = X[:, :41]
f42 = X[:, :42]
f43 = X[:, :43]
f44 = X[:, :44]
f45 = X[:, :45]
f46 = X[:, :46]
f47 = X[:, :47]
f48 = X[:, :48]
f49 = X[:, :49]
f50 = X[:, :50]
f51 = X[:, :51]
f52 = X[:, :52]
f53 = X[:, :53]
f54 = X[:, :54]
f55 = X[:, :55]
#f56 = X[:, :56]
print("check2.1")

#2. Make a mean value for each column vector
f1_bar = mean(f1)
f2_bar = mean(f2)
f3_bar = mean(f3)
f4_bar = mean(f4)
f5_bar = mean(f5)
f6_bar = mean(f6)
f7_bar = mean(f7)
f8_bar = mean(f8)
f9_bar = mean(f9)
f10_bar = mean(f10)
f11_bar = mean(f11)
f12_bar = mean(f12)
f13_bar = mean(f13)
f14_bar = mean(f14)
f15_bar = mean(f15)
f16_bar = mean(f16)
f17_bar = mean(f17)
f18_bar = mean(f18)
f19_bar = mean(f19)
f20_bar = mean(f20)
f21_bar = mean(f21)
f22_bar = mean(f22)
f23_bar = mean(f23)
f24_bar = mean(f24)
f25_bar = mean(f25)
f26_bar = mean(f26)
f27_bar = mean(f27)
f28_bar = mean(f28)
f29_bar = mean(f29)
f30_bar = mean(f30)
f31_bar = mean(f31)
f32_bar = mean(f32)
f33_bar = mean(f33)
f34_bar = mean(f34)
f35_bar = mean(f35)
f36_bar = mean(f36)
f37_bar = mean(f37)
f38_bar = mean(f38)
f39_bar = mean(f39)
f40_bar = mean(f40)
f41_bar = mean(f41)
f42_bar = mean(f42)
f43_bar = mean(f43)
f44_bar = mean(f44)
f45_bar = mean(f45)
f46_bar = mean(f46)
f47_bar = mean(f47)
f48_bar = mean(f48)
f49_bar = mean(f49)
f50_bar = mean(f50)
f51_bar = mean(f51)
f52_bar = mean(f52)
f53_bar = mean(f53)
f54_bar = mean(f54)
f55_bar = mean(f55)
f56_bar = mean(f56)
print("check3")

#3. Make a sigma value for each column vector
f1_std = std(f1)
f2_std = std(f2)
f3_std = std(f3)
f4_std = std(f4)
f5_std = std(f5)
f6_std = std(f6)
f7_std = std(f7)
f8_std = std(f8)
f9_std = std(f9)
f10_std = std(f10)
f11_std = std(f11)
f12_std = std(f12)
f13_std = std(f13)
f14_std = std(f14)
f15_std = std(f15)
f16_std = std(f16)
f17_std = std(f17)
f18_std = std(f18)
f19_std = std(f19)
f20_std = std(f20)
f21_std = std(f21)
f22_std = std(f22)
f23_std = std(f23)
f24_std = std(f24)
f25_std = std(f25)
f26_std = std(f26)
f27_std = std(f27)
f28_std = std(f28)
f29_std = std(f29)
f30_std = std(f30)
f31_std = std(f31)
f32_std = std(f32)
f33_std = std(f33)
f34_std = std(f34)
f35_std = std(f35)
f36_std = std(f36)
f37_std = std(f37)
f38_std = std(f38)
f39_std = std(f39)
f40_std = std(f40)
f41_std = std(f41)
f42_std = std(f42)
f43_std = std(f43)
f44_std = std(f44)
f45_std = std(f45)
f46_std = std(f46)
f47_std = std(f47)
f48_std = std(f48)
f49_std = std(f49)
f50_std = std(f50)
f51_std = std(f51)
f52_std = std(f52)
f53_std = std(f53)
f54_std = std(f54)
f55_std = std(f55)
f56_std = std(f56)
print("check4")

f1_s = (f1 .- f1_bar) ./ f1_std
f2_s = (f2 .- f2_bar) ./ f2_std
f3_s = (f3 .- f3_bar) ./ f3_std
f4_s = (f4 .- f4_bar) ./ f4_std
f5_s = (f5 .- f5_bar) ./ f5_std
f6_s = (f6 .- f6_bar) ./ f6_std
f7_s = (f7 .- f7_bar) ./ f7_std
f8_s = (f8 .- f8_bar) ./ f8_std
f9_s = (f9 .- f9_bar) ./ f9_std
f10_s = (f10 .- f10_bar) ./ f10_std
f11_s = (f11 .- f11_bar) ./ f11_std
f12_s = (f12 .- f12_bar) ./ f12_std
f13_s = (f13 .- f13_bar) ./ f13_std
f14_s = (f14 .- f14_bar) ./ f14_std
f15_s = (f15 .- f15_bar) ./ f15_std
f16_s = (f16 .- f16_bar) ./ f16_std
f17_s = (f17 .- f17_bar) ./ f17_std
f18_s = (f18 .- f18_bar) ./ f18_std
f19_s = (f19 .- f19_bar) ./ f19_std
f20_s = (f20 .- f20_bar) ./ f20_std
f21_s = (f21 .- f21_bar) ./ f21_std
f22_s = (f22 .- f22_bar) ./ f22_std
f23_s = (f23 .- f23_bar) ./ f23_std
f24_s = (f24 .- f24_bar) ./ f24_std
f25_s = (f25 .- f25_bar) ./ f25_std
f26_s = (f26 .- f26_bar) ./ f26_std
f27_s = (f27 .- f27_bar) ./ f27_std
f28_s = (f28 .- f28_bar) ./ f28_std
f29_s = (f29 .- f29_bar) ./ f29_std
f30_s = (f30 .- f30_bar) ./ f30_std
f31_s = (f31 .- f31_bar) ./ f31_std
f32_s = (f32 .- f32_bar) ./ f32_std
f33_s = (f33 .- f33_bar) ./ f33_std
f34_s = (f34 .- f34_bar) ./ f34_std
f35_s = (f35 .- f35_bar) ./ f35_std
f36_s = (f36 .- f36_bar) ./ f36_std
f37_s = (f37 .- f37_bar) ./ f37_std
f38_s = (f38 .- f38_bar) ./ f38_std
f39_s = (f39 .- f39_bar) ./ f39_std
f40_s = (f40 .- f40_bar) ./ f40_std
f41_s = (f41 .- f41_bar) ./ f41_std
f42_s = (f42 .- f42_bar) ./ f42_std
f43_s = (f43 .- f43_bar) ./ f43_std
f44_s = (f44 .- f44_bar) ./ f44_std
f45_s = (f45 .- f45_bar) ./ f45_std
f46_s = (f46 .- f46_bar) ./ f46_std
f47_s = (f47 .- f47_bar) ./ f47_std
f48_s = (f48 .- f48_bar) ./ f48_std
f49_s = (f49 .- f49_bar) ./ f49_std
f50_s = (f50 .- f50_bar) ./ f50_std
f51_s = (f51 .- f51_bar) ./ f51_std
f52_s = (f52 .- f52_bar) ./ f52_std
f53_s = (f53 .- f53_bar) ./ f53_std
f54_s = (f54 .- f54_bar) ./ f54_std
f55_s = (f55 .- f55_bar) ./ f55_std
f56_s = (f56 .- f56_bar) ./ f56_std
print("check5")
X = [f1_s f2_s f3_s f4_s f5_s f6_s f7_s f8_s f9_s f10_s f11_s f12_s f13_s f14_s f15_s f16_s f17_s f18_s f19_s f20_s f21_s f22_s f23_s f24_s f25_s f26_s f27_s f28_s f29_s f30_s f31_s f32_s f33_s f34_s f35_s f36_s f37_s f38_s f39_s f40_s f41_s f42_s f43_s f44_s f45_s f46_s f47_s f48_s f49_s f50_s f51_s f52_s f53_s f54_s f55_s f56_s ]'
#----------------------------------------------
y_hat, decision_values = LIBSVM.svmpredict(model_radialbasis, X)
#-----------------------------------------------------
summary = DataFrame([decision_values[2, :] decision_values[1, :]], :auto)


df_actual_test = CSV.read(testing_dataset, DataFrame) #have to reread or else ID is dropped idk why
submission = DataFrame(Id = df_actual_test[!, :Id], class_0 = summary[!, :x2], class_1 = summary[!, :x1])

CSV.write("offline_submission.csv",submission)




  0.001119 seconds (4.13 k allocations: 150.930 KiB)
  0.000056 seconds (124 allocations: 10.750 KiB)
df_training_clean is loadedcheck1check2check2.1check3check4check5

"offline_submission.csv"

In [68]:
submission

Row,Id,class_0,class_1
,String15,Float64,Float64
1,00eed32682bb,1.0e-7,1.0
2,010ebe33f668,1.0e-7,1.0
3,02fa521e1838,1.0e-7,1.0
4,040e15f562a2,1.0e-7,1.0
5,046e85c7cc7f,1.0e-7,1.0
